In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
os.chdir('/Users/emma.tebbe/berkeley_git/w207_final_project')

In [3]:
data_1 = np.load('data/et_w207_project_npz_files_5000_tmp_tmp5ocrhjnn.npz')
data_1.close
data_2 = np.load('data/et_w207_project_npz_files_5000_tmp_tmp092sag67.npz')
data_2.close
data_3 = np.load('data/et_w207_project_npz_files_5000_tmp_tmpf31_pn8p.npz')
data_3.close
data_4 = np.load('data/et_w207_project_npz_files_5000_tmp_tmpq5b2g4n2.npz')
data_4.close
data_key = np.concatenate((data_1['arr_0'], data_2['arr_0'], data_3['arr_0'], data_4['arr_0']))
data_pixel = np.concatenate((data_1['arr_1'], data_2['arr_1'], data_3['arr_1'], data_4['arr_1']))

df = pd.read_csv('data/school_project_data_set-query.csv')

In [4]:
data_pixel

array([[-103.939    , -116.779    , -123.68     , ..., -103.939    ,
        -116.779    , -123.68     ],
       [ -46.939003 ,  -74.779    , -118.68     , ...,  -57.939003 ,
         -84.779    , -118.68     ],
       [ 151.061    ,  138.22101  ,  131.32     , ...,  151.061    ,
         138.22101  ,  131.32     ],
       ...,
       [  77.061    ,   46.221    ,   32.32     , ...,  -55.939003 ,
         -38.779    ,    1.3199997],
       [-102.939    , -115.779    , -122.68     , ...,  -68.939    ,
         -95.779    , -114.68     ],
       [ 112.061    ,  100.221    ,   89.32     , ...,   -5.939003 ,
          33.221    ,    3.3199997]], dtype=float32)

In [5]:
data_dict = {}
for i in range(data_key.shape[0]):
    data_dict[data_key[i]] = [data_pixel[i]]

df_pixel = pd.DataFrame(data=data_dict)
df_pixel_t = df_pixel.transpose()
df_pixel_t.index.names = ['attachment_key']
df_pixel_t.columns = ['pixel_array']
df_pixel_t

,pixel_array
attachment_key,
ozbpir-9ey6js-ggdqwo,"[-103.939, -116.779, -123.68, -103.939, -116.7..."
378ykanq,"[-46.939003, -74.779, -118.68, -24.939003, -57..."
p366md-big834-7o23k6,"[151.061, 138.22101, 131.32, 151.061, 138.2210..."
3zruwvl2,"[-3.939003, -66.779, -96.68, -3.939003, -66.77..."
q18iae-3vnh74-79npmy,"[-103.939, -116.779, -123.68, -103.939, -116.7..."
...,...
pktyig-481wyg-6gyulf,"[-91.939, -104.779, -97.68, -92.939, -105.779,..."
zu7y3rha,"[-1.939003, -14.778999, -21.68, -1.939003, -14..."
pxcwxf-8tnpyg-8vhb9r,"[77.061, 46.221, 32.32, 79.061, 49.221, 33.32,..."


In [6]:
merge_meta_pixel = df.merge(df_pixel_t, on='attachment_key')
combined_df = merge_meta_pixel[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w',  'pixel_array', 'filename','logo']]
combined_df.head()

,attachment_key,size_mb,height,width,h_to_w,pixel_array,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",1116-100_WGTank_white.jpg,0


In [7]:
file_types = []
for item in combined_df['filename']:
    file_types.append(item[-3:].lower())

print(set(file_types))

#File types should be png, jpg or other
#Need to one hot encode these

{'pig', '0mm', 'ite', 'jpe', '73e', 'jpg', 'peg', 'ack', 'age', 'lue', 'hot', 'png'}


In [8]:
filetype_encoding = []
for item in combined_df['filename']:
    if item[-3:].lower() == 'jpg':
        filetype_encoding.append('jpg')
    elif item[-3:].lower() == 'png':
        filetype_encoding.append('png')
    else:
        filetype_encoding.append('other')

In [9]:
combined_df['filetype'] = filetype_encoding
combined_df = combined_df[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w', 'filetype', 'pixel_array', 'filename','logo']]
combined_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",1116-100_WGTank_white.jpg,0


In [10]:
# get dummies
cork_dummies = pd.get_dummies(combined_df.filetype)

# add to initial df
combined_df = pd.concat((combined_df, cork_dummies), axis=1)

# print head of df
combined_df.head()

,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo,jpg,other,png
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",Esterdale Theatre - Logo.png,1,0,0,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",BLUE_LOGO.png,1,0,0,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",ZEN_LB-5.jpg,0,1,0,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",article central 200.jpg,1,1,0,0
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",1116-100_WGTank_white.jpg,0,1,0,0


In [11]:
combined_df = combined_df[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w', 'jpg', 'png', 'other', 'pixel_array', 'filetype','filename','logo']]
combined_df.head()

,attachment_key,size_mb,height,width,h_to_w,jpg,png,other,pixel_array,filetype,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,0,1,0,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",png,Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,0,1,0,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",png,BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,1,0,0,"[27.060997, 14.221001, 13.32, 23.060997, 13.22...",jpg,ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,1,0,0,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",jpg,article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,1,0,0,"[151.061, 138.22101, 131.32, 151.061, 138.2210...",jpg,1116-100_WGTank_white.jpg,0


In [35]:
combined_df = combined_df[combined_df.notnull()]
combined_df.describe()

,size_mb,height,width,h_to_w,jpg,png,other,logo
count,3755.000000,3754.000000,3754.000000,3754.000000,3755.000000,3755.000000,3755.000000,3755.000000
mean,2.320780,1722.128396,2288.567395,0.758507,0.616245,0.367510,0.016245,0.595473
std,6.304738,1686.929969,2006.283316,0.421649,0.486364,0.482191,0.126433,0.490866
min,0.000226,1.000000,1.000000,0.052055,0.000000,0.000000,0.000000,0.000000
25%,0.038580,454.250000,800.000000,0.514872,0.000000,0.000000,0.000000,0.000000
50%,0.257622,1065.000000,1551.000000,0.666731,1.000000,0.000000,0.000000,1.000000
75%,1.778975,2643.750000,3245.500000,1.000000,1.000000,1.000000,0.000000,1.000000
max,224.959750,15119.000000,16000.000000,4.315143,1.000000,1.000000,1.000000,1.000000


In [55]:
combined_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3755 entries, 0 to 3754
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   attachment_key  3755 non-null   object 
 1   size_mb         3755 non-null   float64
 2   height          3754 non-null   float64
 3   width           3754 non-null   float64
 4   h_to_w          3754 non-null   float64
 5   jpg             3755 non-null   uint8  
 6   png             3755 non-null   uint8  
 7   other           3755 non-null   uint8  
 8   pixel_array     3755 non-null   object 
 9   filetype        3755 non-null   object 
 10  filename        3755 non-null   object 
 11  logo            3755 non-null   int64  
dtypes: float64(4), int64(1), object(4), uint8(3)
memory usage: 304.4+ KB


In [68]:
#Feature is just the pixel array
X = np.array(combined_df.iloc[:3755,8].tolist())
print(X[0])
print(X.shape)
print(np.count_nonzero(np.isnan(X)))

[151.061   138.22101 131.32    ... 151.061   138.22101 131.32   ]
(3755, 150528)
0


In [69]:
#Features are size, height, width, h_to_w
X2 = combined_df.iloc[:3755, 1:5].values
print(X2[0])
print(X2.shape)
print(np.count_nonzero(np.isnan(X2)))

[1.184900e-02 5.200000e+02 5.060000e+02 1.027668e+00]
(3755, 4)
3


In [70]:
#Features are size, height, width, h_to_w, pixel_array
X3 = np.concatenate((X2,X), axis=1)
print(X3.shape)
print(X3[0][0])
print(np.count_nonzero(np.isnan(X3S)))

(3755, 150532)
0.011849
0


In [66]:
#Features are size, height, width, h_to_w, filetype
X4 = combined_df.iloc[:3755, 1:8].values
print(X4[0])
print(X4.shape)

[1.184900e-02 5.200000e+02 5.060000e+02 1.027668e+00 0.000000e+00
 1.000000e+00 0.000000e+00]
(3755, 7)


In [67]:
#Features are size, height, width, h_to_w, filetype, pixel array
X5 = np.concatenate((X4,X), axis=1)
print(X5.shape)
print(X5[0][0])

(3755, 150535)
0.011849


In [71]:
Y = np.array(combined_df['logo'])
print(Y.shape)

(3755,)


In [39]:
# TO KEEP TRACK
# Y - logo / not logo
# X - just pixel array
# X2 - size, height, width, h_to_w
# X3 - size, height, width, h_to_w, pixel_array
# X4 - size, height, width, h_to_w, filetype
# X5 - size, height, width, h_to_w, filetype, pixel array

In [ ]:
ss1 = StandardScaler()
XS = ss1.fit_transform(X)

ss2 = StandardScaler()
X2S = ss2.fit_transform(X2)

ss3 = StandardScaler()
X3S = ss3.fit_transform(X3)

ss4 = StandardScaler()
X4S = ss3.fit_transform(X4)

ss5 = StandardScaler()
X5S = ss3.fit_transform(X5)

In [53]:
# Having issues with NaN values in X2, so removing those 3 data points from all versions
print(np.count_nonzero(np.isnan(X2S)))

# XS= XS[-np.argwhere(np.isnan(X2S))]
# X2S= X2S[-np.argwhere(np.isnan(X2S))]
# X3S= X3S[-np.argwhere(np.isnan(X2S))]
# X4S= X4S[-np.argwhere(np.isnan(X2S))]
# X5S= X5S[-np.argwhere(np.isnan(X2S))]

print(np.count_nonzero(np.isnan(X3S)))
print(np.count_nonzero(np.isnan(X4S)))
print(np.count_nonzero(np.isnan(X5S)))

3
0
0
0


In [ ]:
pca1 = PCA(n_components=2)
x_pca1 = pca1.fit_transform(XS)


In [ ]:
pca2 = PCA(n_components=2)
x_pca2 = pca2.fit_transform(X2S)

pca3 = PCA(n_components=2)
x_pca3 = pca3.fit_transform(X3S)

pca4 = PCA(n_components=2)
x_pca4 = pca4.fit_transform(X4S)

pca5 = PCA(n_components=2)
x_pca5 = pca5.fit_transform(X5S)

In [ ]:
colors = ['r' if label == 0 else 'g' for label in Y]

In [ ]:
x = [item[0] for item in x_pca1] 
y = [item[1] for item in x_pca1] 

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
ax.scatter(x, y, c = colors, s = 5)
ax.grid()